# Advanced Text Analytics Lab 2

This notebook is the second of two lab notebooks that you will submit as part of your assessment for the Advanced Data Analytics unit. The notebook contains three required sections plus an optional section:

4. **Introducing Transformers:** This section introduces the Transformers library from HuggingFace, showing you how to use it to obtain contextualised embeddings from pretrained transformer models.

5. **Question Answering with Pretrained Transformers:** Learn about how to use a pretrained model to perform automatic question answering. 

6. **Transformers for Text Classification:** Here we show you how to construct a classifier using Transformers.

7. **OPTIONAL: More on Transformers:** Some pointers to other materials if you want to learn more about transformers, e.g., if using them in your summer project. 

Example code for all the tasks has been tested on a three-year old MacBook Pro, and the longest training process took under 10 minutes. If you find that the code takes too long to run on your own machine, you can try [Google Colab](https://colab.research.google.com/), Amazon Sagemaker Studio, or use lab machines on campus provided by the school. 

## Learning Outcomes

These sections will contain tutorial-like instructions, as you have seen in previous text analytics labs. On completing these sections, the intended learning outcomes are that you will be able to...
1. Use pretrained transformers to obtain contextualised word and sentence embeddings.
1. Apply a pretrained QA model to a new dataset. 
1. Construct classifiers with pretrained transformers. 
1. Find documentation on pretrained models in the Transformers library.

## Your Tasks

Inside each of these sections there are several **'To-do's**, which you must complete for your summative assessment. Your marks will be based on your answers to these to-dos. Please make sure to:
1. Include the output of your code in the saved notebook. Plots and printed output should be visible without re-running the code. 
1. Include all code needed to generate your answers.
1. Provide sufficient comments to understand how your method works.
1. Write text in a cell in markdown format where a written answer is required. You can convert a cell to markdown format by pressing Escape-M. 

There are also some unmarked 'to-do's that are part of the tutorial to help you learn how to implement and use the methods studied here.

## Good Academic Practice

Please follow [the guidance on academic integrity provided by the university](http://www.bristol.ac.uk/students/support/academic-advice/academic-integrity/).
You are required to write your own answers -- do not share your notebooks or copy someone else's writing. Do not copy text or long blocks of code directly into the notebook from online sources -- always rewrite in your own way. Breaking the rules can lead to strong penalties. 

## Marking Criteria

1. The coursework (both notebooks) is worth 30% of the unit in total. 
1. There is a total of 100 marks available for both lab notebooks. 
1. This notebook is worth 50 of those marks.
1. The number of marks for each to-do out of 100 is shown alongside each to-do.
1. For to-dos that require you to write code, a good solution would meet the following criteria (in order of importance):
   1. Solves the task or answers the question asked in the to-do. This means, if the code cells in the notebook are executed in order, we will get the output shown in your notebook.
   1. The code is easy to follow and does not contain unnecessary steps.
   1. The comments show that you understand how your solution works.
   1. A very good answer will also provide code that is computationally efficient but easy to read.
1. You can use any suitable publicly available libraries. Unless the task explicitly asks you to implement something from scratch, there is no penalty for using libraries to implement some steps.

## Support

The main source of support will be during the remaining lab sessions (Fridays 3-6pm) for this unit. 

The TAs and lecturer will help you with questions about the lectures, the code provided for you in this notebook, and general questions about the topics we cover. For the marked 'to-dos', they can only answer clarifying questions about what you have to do. 

Office hours: You can book office hours with Edwin on Mondays 3pm-5pm by sending him an email (edwin.simpson@bristol.ac.uk). If those times are not possible for you, please contact him by email to request an alternative. 

## Deadline

The notebook must be submitted along with the second notebook on Blackboard before **Wednesday 24th May at 13.00**. 

## Submission

You will need to zip up this notebook with the previous notebook into a single .zip file, which you will submit to Blackboard through the 'assessment, submission and feedback' link on the left sidebar. 

Please name your files like this:
   * Name this notebook ADA2_<student_number>.ipynb
   * Name the zip file <student_number>.zip
   * Please don't use your name anywhere as we want to mark anonymously. 

In [44]:
import numpy as np
import torch 
from datasets import load_dataset
from sklearn.metrics import classification_report

cache_dir = "./data_cache"

# 4. Pretrained Transformers (max. 15 marks)

HuggingFace is a company that has developed an open source library for loading pretrained transformer models. They also distribute many models that have been pretrained using language modelling tasks, or fine-tuned to specific downstream NLP tasks.  It is currently the best library to use to create NLP models on top of large, deep neural networks. This is especially useful for tasks where simpler, feature-based methods or smaller LSTM models do not perform well enough, for example, when complex processing of syntax and semantics is required (natural language 'understanding'). 

Let's start by looking at two key types of object in the transformers library: models and tokenizers.

## 4.1. Models

The neural network models available in the Transformers library are accessed through wrapper classes such as `AutoModel`. If we want to load a pretrained model, we can simply pass its name to the `from_pretrained` function, and the pretrained model weights will be downloaded from HuggingFace and a neural network model will be created with those weights. For example:

In [45]:
from transformers import AutoModel # For BERTs

model = AutoModel.from_pretrained("huawei-noah/TinyBERT_General_4L_312D") 

Some weights of the model checkpoint at huawei-noah/TinyBERT_General_4L_312D were not used when initializing BertModel: ['fit_denses.0.bias', 'fit_denses.3.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'fit_denses.2.bias', 'fit_denses.1.bias', 'fit_denses.0.weight', 'fit_denses.4.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'fit_denses.2.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'fit_denses.4.weight', 'fit_denses.3.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'fit_denses.1.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing

This code loads the TinyBERT model, which is a compressed version of BERT. It has 4.4 million parameters, compared to the standard version of BERT, 'BERT-base', which has 110 million parameters. While TinyBERT will not perform as well as larger models, we will use it for this notebook to save memory and computation costs. See [documentation here](https://huggingface.co/huawei-noah/TinyBERT_General_4L_312D).

<!--the RoBERTa variant of BERT. It has 4.4 million parameters, compared to the standard version of BERT, 'BERT-base', which has 110 million parameters. While RoBERTa-tiny will not perform as well as larger models, we will use it for this notebook to save memory and computation costs. See [documentation here](https://huggingface.co/arampacha/roberta-tiny).  -->

The same functions can be used to load other models from HuggingFace's repository simply by changing the model's name. Take a look at [the Models page](https://huggingface.co/models) so see what there is on offer. Do you recognise any of the models' names?

# 4.2. Tokenizers

Before we can apply a model to some text, we need to a create Tokenizer object. In Transfomers, Tokenizer objects convert raw text to a sequence of numbers. First, the tokenizer actually performs tokenization, then it maps each token to its numerical ID. There are lots of different tokenizers that we can use to preprocess text. If we are loading a pretrained model, we will need to choose the tokenizer that corresponds to that model. 

We can load the right tokenizer as follows, in the same way we loaded the model itself:

In [46]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")

Let's see what the TinyBERT tokenizer does to an example sentence:

In [47]:
sentence = "The transformer architecture has transformed the field of NLP."

tokens = tokenizer.tokenize(sentence)
print(tokens)

['the', 'transform', '##er', 'architecture', 'has', 'transformed', 'the', 'field', 'of', 'nl', '##p', '.']


Let's compare with the NLTK tokenizer we have seen before:

In [48]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

nltk_tokens = word_tokenize(sentence)
print(nltk_tokens)

['The', 'transformer', 'architecture', 'has', 'transformed', 'the', 'field', 'of', 'NLP', '.']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\antro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


While NLTK keeps whole words as tokens, the BERT tokenizer splits some words into sub-words and inserts some special characters into the tokens. Splitting is applied to words with low frequency in the training set, such as 'transformer'. 

**TO-DO 4a:** What is the benefit of splitting rare words into sub-word tokens? **(2 marks)**

WRITE YOUR ANSWER HERE:

It can handle rare and unknown words better by breaking down words it has not seen during training into smaller sub-word units (that it may have seen). This can also reduce vocabulary size as it can keep tokens for the more common sub-words that make up bigger words. It doesn't then have to keep a token for every possible word in the language.
---

It is important to use the right tokenizer with a pretrained model as each model was trained with text tokenized in a particular way. After tokenization, the Tokenizer object can also map the tokens to their IDs (indexes in the vocabulary):

In [49]:
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

[1996, 10938, 2121, 4294, 2038, 8590, 1996, 2492, 1997, 17953, 2361, 1012]


## 4.3. Contextualised Embeddings

Now that we have a sequence of tokens, we are almost ready to process the sequence using the pretrained model. 

Our model takes as input a PyTorch `tensor` object. In PyTorch, `tensor` is a muli-dimensional matrix. Here, we need a two-dimensional matrix, where each row is a sequence of input tokens corresponding to a single sentence or document. Let's convert our list of IDs to a 2-D tensor with a single row:

In [50]:
ids_tensor = torch.tensor([ids])

print(ids_tensor)

tensor([[ 1996, 10938,  2121,  4294,  2038,  8590,  1996,  2492,  1997, 17953,
          2361,  1012]])


Now we can process the sequence using our model. The model maps the sequence of input IDs to a sequence of output vectors, which are contextualised word embeddings. The hidden state values produced in the last hidden layer of the model are used as the contextualised embeddings:

In [51]:
model_outputs = model(ids_tensor)
print('The complete model outputs: ')
print(model_outputs)

print()
print('The last hidden state for the first token in the sequence (the first word embedding): ')
embeddings = model_outputs['last_hidden_state'][0]
print(embeddings)

The complete model outputs: 
BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.3608,  0.2862, -0.1549,  ..., -0.2064,  0.2663, -0.0109],
         [ 0.0149,  0.7223, -0.0508,  ..., -0.5505,  0.2355, -0.2962],
         [ 0.1531,  0.5903, -0.1244,  ..., -0.4263,  0.0417, -0.1839],
         ...,
         [ 0.1742, -0.1091, -0.1963,  ..., -0.6736,  0.0472, -0.1840],
         [ 0.2434,  0.1021, -0.2241,  ..., -0.5400, -0.1691, -0.1314],
         [ 0.0854,  0.3272, -0.3016,  ..., -0.2154, -0.5632, -0.1921]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-1.1380e-02, -6.3006e-03,  1.8521e-02,  7.1139e-03, -3.1795e-02,
          1.3882e-02, -1.5459e-02, -1.0610e-03, -1.8263e-02, -3.6515e-02,
         -2.1257e-02, -1.5479e-02, -2.8088e-04, -4.1092e-02, -2.5315e-02,
         -4.3338e-02, -1.1616e-03, -1.3931e-02,  6.0733e-03,  4.3790e-03,
          2.7084e-04, -2.1810e-02, -4.8026e-02,  2.5493e-02, -1.6502e-02,
         -1.2034e-03,  4.2757e-02,  3.

We can retrieve the embedding vector for "transform" like this ("transform" is the second token in the sequence):

In [52]:
emb = embeddings[1]  # get second embedding in the sequence

# convert it to a numpy array so we can perform various operations on it later on
emb = emb.detach().numpy()

print(emb)
print(f'The TinyBERT embeddings have {emb.shape[0]} dimensions.')

[ 1.49153005e-02  7.22317874e-01 -5.07868230e-02 -2.74204463e-01
 -1.38932630e-01  1.00099564e+00  7.11465254e-03  2.71391183e-01
 -3.92820872e-02  6.04104064e-02  1.25739962e-01  4.60631520e-01
  6.25199080e-03  1.61929354e-01  1.23913094e-01 -4.08095986e-01
  1.24867618e-01 -4.71536130e-01  2.24769622e-01  6.35194033e-02
  8.56170431e-02 -1.88045308e-01  1.77257478e-01  3.40048701e-01
 -1.95546389e-01  1.58553630e-01  9.62862372e-02  1.12649322e-01
  2.21045420e-01 -9.56113696e-01 -3.85949075e-01  1.39220759e-01
  5.90012372e-01 -8.06728721e-01 -1.34287700e-01  2.35691726e-01
 -1.02274746e-01  2.78304130e-01  7.94321060e-01 -2.49363124e-01
  1.72772780e-01 -2.07583055e-01  3.00157607e-01 -8.59335139e-02
 -2.25285500e-01 -9.75413024e-02 -3.52349013e-01  3.81161451e-01
 -3.87680352e-01 -1.77613348e-01 -4.13685799e-01  1.38046905e-01
  1.29872635e-02  6.52685225e-01  1.16503887e-01 -5.10779381e-01
 -8.30414295e-02 -2.67042965e-02  3.12862992e-01 -2.62848735e-01
 -1.43284842e-01  1.10269

In [53]:
print(tokens[1])
print(tokens[3])

transform
architecture


TO-DO 4b: Retrieve the embedding for "architecture" (this to-do will not be marked).

In [54]:
# WRITE YOUR ANSWER HERE
emb = embeddings[3]  # get second embedding in the sequence

# convert it to a numpy array so we can perform various operations on it later on
emb = emb.detach().numpy()

print(emb)

[ 2.71386713e-01  7.74581492e-01 -3.24256897e-01 -7.14321211e-02
 -4.96365130e-04  9.37309921e-01 -4.40283399e-03 -4.26924862e-02
  1.27398055e-02  1.89262237e-02  1.02528319e-01  4.54656303e-01
  2.70435780e-01  2.30988353e-01  4.03703563e-03 -1.08994454e-01
 -4.59913611e-02 -3.51154804e-01 -1.34710267e-01  8.29393715e-02
  1.86497152e-01  5.00279889e-02  7.21666664e-02  2.28657633e-01
 -2.19697639e-01  9.40192565e-02  1.65540621e-01  1.85794771e-01
  3.17783445e-01 -5.09367108e-01 -5.00949264e-01  1.52487487e-01
  4.57998574e-01 -8.51875722e-01 -1.58632293e-01  1.58964545e-01
  4.16199565e-02  2.30998576e-01  8.78503442e-01 -6.23165891e-02
  1.87219307e-01 -1.23371184e-02  2.10083604e-01  3.48072872e-02
 -2.51239657e-01 -1.37914494e-01 -3.88696969e-01  2.98189402e-01
 -2.92033911e-01 -3.19504052e-01 -1.98435426e-01  1.32031590e-01
 -6.46382272e-02  7.43182957e-01  7.14244917e-02 -3.02118927e-01
  3.49780679e-01 -5.81787825e-02  2.85069287e-01 -4.09581006e-01
 -1.03296980e-01  1.03767

Sentences and documents usually have varying lengths. So, to put multiple sentences into a single tensor, we need to pad the sequences up to a maximum length. Luckily, the tokenizer class takes care of this for us. When we pass in a list of sentences, the tokenizer creates a matrix, where each row is a sequence of the same length:

In [55]:
sentences = [
    "They received a loan from the bank.",
    "It was not good for either his bank balance or his blood pressure.",
    "She walked along the bank of the river towards the city.",
    "They bank their cheques on Thursdays.",
    "She walked along the embankment towards the city."
]

model_inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")  

print(model_inputs)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[  101,  2027,  2363,  1037,  5414,  2013,  1996,  2924,  1012,   102,
             0,     0,     0,     0,     0,     0],
        [  101,  2009,  2001,  2025,  2204,  2005,  2593,  2010,  2924,  5703,
          2030,  2010,  2668,  3778,  1012,   102],
        [  101,  2016,  2939,  2247,  1996,  2924,  1997,  1996,  2314,  2875,
          1996,  2103,  1012,   102,     0,     0],
        [  101,  2027,  2924,  2037, 18178, 10997,  2006,  9432,  2015,  1012,
           102,     0,     0,     0,     0,     0],
        [  101,  2016,  2939,  2247,  1996, 22756,  2875,  1996,  2103,  1012,
           102,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': t

`model_inputs` is a dictionary containing three objects:
 * The `input_ids` are the list of token IDs in the input sequences. 
 * The `attention_mask` records which tokens are special padding tokens and which are real tokens. Tokens with a 0 in the attention mask will be ignored.
 * `token_type_ids` is needed when two sequences are passed together as input to the model for tasks such as next sentence prediction that involve comparing two sentences. Here, each input is a single sentence, so we have only one type of token in the output above. 
 
TO-DO 4c: What value do the special padding tokens have? (this to-do is unmarked)

ANSWER: 0 (zero)

---

Notice that the input_ids all start with the same token ID, 101, even though they have different first words. They also have token ID 102 before the padding tokens. This is because the tokenizer inserts two special tokens, which are used in some applicaions of BERT. 101 is the '[CLS]' token, which is a dummy token whose embedding can be trained to represent the whole sequence. the [CLS] token's embedding can then be used as input to a text classifier to classify a sentence or document. Token 102 is '[SEP]', which can be used to separate multiple input sequences in a single example. This is needed in tasks where multiple pieces of text are provided as input, e.g., a to build a classifier that can determine whether two sentences contradict each other. 

We can now pass all of the model inputs to the model to produce a set of contextualised embeddings:

In [56]:
# model_inputs is a dictionary, so to provide the arguments to model(), 
# we use the double star to unpack the dictionary so that each key in the dictionary is
# an argument to model() and each value is the value of the argument. 
model_outputs = model(**model_inputs) 

In [57]:
print(model_inputs[0])

Encoding(num_tokens=16, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [58]:
print(model_inputs[0].tokens)

['[CLS]', 'they', 'received', 'a', 'loan', 'from', 'the', 'bank', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [59]:
print(model_inputs[4].tokens)

['[CLS]', 'she', 'walked', 'along', 'the', 'embankment', 'towards', 'the', 'city', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


**TO-DO 4d:** The first four example sentences above all contain the word "bank", and the last example contains "embankment". Obtain a list of contextualised word embeddings for 'bank' and 'embankment' in the example sentences using our model. **(4 marks)**

Hint: you may need to convert tensors to numpy arrays.

In [60]:
# tokenize the sentences
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

# forward pass through the model (as above with the **)
outputs = model(**inputs)

# obtain the output embeddings (
embeddings = outputs.last_hidden_state

# tokenize 'bank' and 'embankment'
bank_ids = tokenizer.encode('bank', add_special_tokens=False)
embankment_ids = tokenizer.encode('embankment', add_special_tokens=False)

# indices of 'bank' and 'embankment'
bank_indices = [[i for i, token_id in enumerate(input_ids) if token_id in bank_ids] for input_ids in inputs['input_ids']]
embankment_indices = [[i for i, token_id in enumerate(input_ids) if token_id in embankment_ids] for input_ids in inputs['input_ids']]

# get the embedings for 'bank' and 'embankment' 
bank_embs = [[embedding[token_idx].detach().numpy() for token_idx in indices] for embedding, indices in zip(embeddings, bank_indices)]
embankment_embs = [[embedding[token_idx].detach().numpy() for token_idx in indices] for embedding, indices in zip(embeddings, embankment_indices)]

print("Bank embs:")
for i, emb in enumerate(bank_embs):
    if emb:  # check not empty
        print(f"Sentence {i + 1}: {emb}")

print("\nEmbankment embs:")
for i, emb in enumerate(embankment_embs):
    if emb:  
        print(f"Sentence {i + 1}: {emb}")

Bank embs:
Sentence 1: [array([ 0.52827066, -0.04592289,  0.1308929 , -0.15328005, -0.16961122,
       -0.06582491, -0.5669055 ,  0.33207503,  0.23153198,  0.00656659,
       -0.19418658, -0.5983134 , -1.077471  ,  0.8210215 , -0.21299282,
        0.32614294,  0.9019877 ,  0.37719253, -0.33868232, -0.05000402,
        0.03580145,  0.48318177, -1.219599  ,  0.2654366 ,  0.1540078 ,
        0.35209602,  0.70070946, -0.20933267, -0.44881603, -0.47062862,
       -0.46232772, -0.13776326, -0.00225069,  0.16826689,  0.10031743,
       -0.01842458,  0.37044448,  0.30816612,  0.5140236 ,  0.7347502 ,
       -0.03685041, -0.7789111 ,  0.18751723,  0.45820737, -0.9271034 ,
        0.37546998,  0.8562455 ,  0.5174336 , -0.6091915 , -0.31556815,
        0.24388759,  0.15135092, -0.1580156 ,  0.01617955,  0.186916  ,
       -0.44020078, -0.75819564,  0.26439977, -0.7627953 ,  0.03252312,
        0.11703229, -0.30380097,  0.00834826, -0.45045626,  0.18094605,
       -0.2271295 , -0.23015597,  0.1776

**TO-DO 4e:** Compute the similarities between these embeddings in the cell below, and show the results. Which embeddings are most similar to one another and why? **(6 marks)**

WRITE YOUR ANSWER HERE:


In [61]:
# WRITE YOUR OWN CODE HERE
from sklearn.metrics.pairwise import cosine_similarity

# combine all the embeddings to create matrix
bank_emb_flat = [emb for sent_emb in bank_embs for emb in sent_emb]
embankment_emb_flat = [emb for sent_emb in embankment_embs for emb in sent_emb]

all_embs = bank_emb_flat + embankment_emb_flat
print(len(all_embs))

similarity_matrix = cosine_similarity(all_embs) # 1 = same, -1 = opposite

# Print the similarity matrix
for i, similarity_row in enumerate(similarity_matrix):
    print(f"Emb {i + 1} similarity: {similarity_row}")


5
Emb 1 similarity: [1.0000001  0.63313466 0.48778638 0.52329594 0.36788744]
Emb 2 similarity: [0.63313466 0.9999999  0.43572867 0.4705648  0.2787157 ]
Emb 3 similarity: [0.48778638 0.43572867 0.9999997  0.4787853  0.7245258 ]
Emb 4 similarity: [0.52329594 0.4705648  0.4787853  0.99999994 0.3186101 ]
Emb 5 similarity: [0.36788744 0.2787157  0.7245258  0.3186101  1.0000002 ]


In [62]:
# It looks like sentences from embeddings 3 and 5 are the most similar at 72%
embedding_numbers = [2, 4] 
all_sentences = [sent for sent in sentences if 'bank' in sent or 'embankment' in sent]

for num in embedding_numbers:
    #print(all_embeddings[num])
    print(f"Original sentence: {all_sentences[num]}")
    print("\n")

Original sentence: She walked along the bank of the river towards the city.


Original sentence: She walked along the embankment towards the city.




**TO-DO 4f:** Use the [CLS] token's embedding to find the most similar **sentence** to "She walked along the embankment towards the city." from the first four sentences. Print the similarities and the selected sentence. **(3 marks)**

In [63]:
# WRITE YOUR OWN CODE HERE

# get the CLS
cls_embs = embeddings[:, 0, :].detach().numpy()

# calculate cosine similarities
similarities = cosine_similarity([cls_embs[-1]], cls_embs[:-1])[0]

# print 
for i, sim in enumerate(similarities, 1):
    print(f"Similarity with {i}: {sim}")

# find the index of most similar sentence
most_similar_idx = similarities.argmax()

# print most similar
print("\nmost similar sentence is:")
print(sentences[most_similar_idx])

Similarity with 1: 0.9093040823936462
Similarity with 2: 0.793634831905365
Similarity with 3: 0.9947959184646606
Similarity with 4: 0.8988914489746094

most similar sentence is:
She walked along the bank of the river towards the city.


# 5. Question Answering with Pretrained Transformers (max. 11 marks)

The previous section showed us how to obtain a sequence of contextualised word embeddings using a pretrained transformer. How are these embeddings used to extract answers from documents to a given question?

First, let's load up the [Tweet QA](https://huggingface.co/datasets/tweet_qa) dataset, which we will use to test a pretrained question answering (QA) model. This dataset contains tweets along with questions about the information in the tweets, and a list of correct answers. As we are not going to train our own QA model (it requires a lot of compute time), we will only need the validation set:

In [64]:
from sklearn.metrics import f1_score

val_dataset = load_dataset(
    "tweet_qa",
    split="validation",
    ignore_verifications=True,
    cache_dir=cache_dir,
)
print(f"Validation dataset with {len(val_dataset)} instances loaded")

C:\Users\antro\anaconda3\envs\data_analytics\lib\site-packages\datasets\load.py:1748: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
Found cached dataset tweet_qa (C:/Users/antro/OneDrive - University of Bristol/ADVANCED DA/Assignment/advanced-labs-public/data_cache/tweet_qa/default/1.0.0/7d588f7f477946b10f60c035ca55175737315ac446102b015218af38d2638777)


Validation dataset with 1086 instances loaded


Now we are working with complete dataset using the HuggingFace datasets library. In the next cell, we create a tokenizer to tokenize the examples in the dataset. We need to choose the right tokenizer for the QA model we want to use, so let's decide to use `"distilbert-base-cased-distilled-squad"` as our pretrained model. This is based on a smaller version of BERT, called Distilbert, which was fine-tuned on the SQUAD question answering dataset.

In [65]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased-distilled-squad") 

def tokenize_function(dataset):
    # Pass two strings to the tokenizer -- it will concatenate them with a [SEP] special token between them. 
    model_inputs = tokenizer(dataset['Question'], dataset['Tweet'], padding="max_length", max_length=200, truncation='only_second')
    return model_inputs

Again, we can use the `map()` method to apply the tokenizer to each example in the dataset. 

In [66]:
val_dataset = val_dataset.map(tokenize_function, batched=True) 

Loading cached processed dataset at C:\Users\antro\OneDrive - University of Bristol\ADVANCED DA\Assignment\advanced-labs-public\data_cache\tweet_qa\default\1.0.0\7d588f7f477946b10f60c035ca55175737315ac446102b015218af38d2638777\cache-6ed8918a1aa971f3.arrow


The type of QA model we are going to work with is _extractive_, meaning that the model will extract the answer from the 'context' (also known as the 'passage' or 'source document'). It does this by identifying the index of the start and end tokens of the answer span within the context, or returning `(0, 0)` (the index 0 for both the start and end token) if the context does not contain an answer to the given question. 

As explained in the lectures, BERT forms the basis of the QA model, and maps each token to a contextualised embedding. The QA model then maps each token's contextualised embedding to the probability that the token is the start of the answer span, and to the probability that the token is the end of the answer span. The layers that map the embeddings to the start and end probabilities are known as the 'head' of the model. [The original BERT paper](https://arxiv.org/pdf/1810.04805.pdf) depicts the QA model like this (Devlin et al., 2018):

<img src="bert_qa.png" alt="BERT QA diagram from the slides in week 10 showing the embedding of each token connected to the start and end output layers" width="400px"/>

We can see a similar structure in most neural network models. Our original text classifier from the first notebook used a fully-connected layer to produce a hidden representation of the whole sentence (rather than using BERT to produce a sequence of embeddings). This hidden representation was then fed to an output layer to produce a probability distribution over class labels (rather than the start and end probabilities):

<img src="neural_text_classifier_smaller.png" alt="Neural text classifier diagram from the slides in lecture 8.1" width="400px"/>


<!--With transformers, 
we can do something very similar, by connecting the transfomer's output to a fully-connected layer. However, with BERT, we do not need to pass the embedding of each individual word to the fully-connected layer because there is a special [CLS] token that represents the whole sentence:

The code below shows how to access a tensor containing the [CLS] embeddings:-->

Now, we have the dataset in the right format, let's see how to load a pretrained QA model based on a pretrained transformer. The QA model was trained by taking a pretrained BERT model (pretrained on masked language modelling with unlabelled text), adding the QA head, then further training the complete model on a QA dataset. 

The transformers library provides some useful wrapper classes for loading pretrained models for various NLP tasks, such as QA or text classification. These 'auto' classes are documented here: https://huggingface.co/docs/transformers/model_doc/auto . Let's use an auto class to load the `"distilbert-base-cased-distilled-squad"` pretrained QA model (this code will try to reload the model from a cache or download the model from HuggingFace):

In [67]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad")

As our model was pretrained, we can use it directly on our Tweet_QA dataset (you may see a message to this effect when you run the cell above the first time). 

So, how do we get a prediction from the model? Let's take a single example from Tweet_QA and obtain the start and end probabilities for all tokens in the 'context':

In [68]:
def predict_nn(qa_model, dataset):
    
    # Switch off dropout
    qa_model.eval()

    # Pass the required inputs from the dataset to the model    
    output = qa_model(attention_mask=torch.tensor(dataset["attention_mask"]), input_ids=torch.tensor(dataset["input_ids"]))
        
    # the output dictionary contains logits, which are the unnormalised scores for each class for each example:
    probs_start = torch.nn.Softmax(dim=1)(output["start_logits"]).detach().numpy()
    probs_end = torch.nn.Softmax(dim=1)(output["end_logits"]).detach().numpy()
        
    return probs_start, probs_end

# Run the prediction function to get the results for the first 20 examples:
probs_start, probs_end = predict_nn(model, val_dataset[0:20])

Now that we have the probabilities that each token is a start or end token, we combine these probabilities to estimate the probability of each possible answer span. This will allow us to choose the answer span with highest probability. 

In the next cell is our first attempt, which you will need to improve to get valid answers. This code loops through each possible combination of start and end tokens, obtains the start and end probabilities, and extracts the answer text for the corresponding span.

**TO-DO 5a:** Use the start and end probabilities to compute the answer span probability at the place marked inside the predict_answer() function below. **2 marks**

In [69]:
# our example:
example_index = 3

example = val_dataset[example_index]
print(f'CONTEXT = {example["Tweet"]}')
print(f'QUESTION = {example["Question"]}')
print(f'LIST OF POSSIBLE ANSWERS = {example["Answer"]}')

CONTEXT = The #endangeredriver would be a sexy bastard in this channel if it had water. Quick turns. Narrow. (I'm losing it) John D. Sutter (@jdsutter) June 21, 2014
QUESTION = what hashtag was used?
LIST OF POSSIBLE ANSWERS = ['#endangeredriver', '#endangereddriver']


In [70]:
def predict_answer(probs_start, probs_end, input_ids, tokenizer):
    
    input_length = len(input_ids)  # length of the input sequence, in the form "[CLS] question [SEP] context"

    SEP_SPECIAL_TOKEN = 102  # the input id for the sep special token the separates the question from the context. The context starts after this token. 
    PAD_SPECIAL_TOKEN = 0  # the input id for padding tokens added to the end of the context
    
    span_probabilities = []  # save the probabilities here
    spans = []  # save the possible answer spans here
    
    for start_idx in range(0, input_length):
        for end_idx in range(0, input_length):
            
            start_prob = probs_start[start_idx]
            end_prob = probs_end[end_idx]
            
            ### WRITE YOUR ANSWER HERE
            
            # answer span probability
            span_prob = start_prob * end_prob
            span_probabilities.append(span_prob)
            
            ###
            
            span = tokenizer.decode(input_ids[start_idx:end_idx+1])
            spans.append(span)

    # sort the spans according to probability:
    sorted_span_index = np.argsort(span_probabilities)
    
    # print the top 20 answers:
    for i in range(20):
        print(f'Span prob = {span_probabilities[sorted_span_index[-i-1]]}, answer = {spans[sorted_span_index[-i-1]]}')
            
predict_answer(probs_start[example_index], probs_end[example_index], example['input_ids'], tokenizer)

Span prob = 0.5619066953659058, answer = endangeredriver
Span prob = 0.15243832767009735, answer = The # endangeredriver
Span prob = 0.12783236801624298, answer = # endangeredriver
Span prob = 0.043352581560611725, answer = 
Span prob = 0.018347155302762985, answer = endangeredriver would be a sexy bastard in this channel if it had water. Quick turns. Narrow
Span prob = 0.014454033225774765, answer = 
Span prob = 0.007982275448739529, answer = endangeredriver would be a sexy bastard in this channel if it had water. Quick turns
Span prob = 0.007700360845774412, answer = endangeredriver would be a sexy bastard in this channel if it had water
Span prob = 0.007665030658245087, answer = 
Span prob = 0.006627378053963184, answer = ##river
Span prob = 0.005845808424055576, answer = 
Span prob = 0.004977356176823378, answer = The # endangeredriver would be a sexy bastard in this channel if it had water. Quick turns. Narrow
Span prob = 0.004173932131379843, answer = # endangeredriver would be a

Are all of the top 20 valid and unique answers? If not, what do you think is going wrong? 

**TO-DO 5b:** Use the cell below to define a new and improved version of `predict_answer()` that only includes valid answers. Summarise in a couple of sentences what kind of invalid answers your code removes. **4 marks**

WRITE YOUR ANSWER HERE:
## The results show overlapping spans, some spans show more than once and some spans are blank (or contain blanks). Some of the spans overlap. They can be nested within one another, e.g. we can see "# endangeredriver would be a sexy bastard in this channel if it had water" and "The # endangeredriver would be a sexy bastard in this channel if it had water" which are nested, and both of which have "# endangeredriver" in them. The model considers all possible spans between each start and end index.

## We can modify the code to remove the blank "" and all whitespace spans, which may happen if the span is in the zero padding area. We can also check for duplicate spans by creating a span store and checking for the existence of the new span before adding it to spans. Have also removed spans which are a single character e.g. # although this may not always be desirable e.g. if the answer could be a single digit number.


## To remove the longer nested spans, we could remove the longer spans that contain shorter nested spans within them, where these shorter spans have a higher probability. Can loop through checking each span against the others.


In [71]:
def predict_answer(probs_start, probs_end, input_ids, tokenizer):
    
    input_length = len(input_ids)  # length of the input sequence, in the form "[CLS] question [SEP] context"

    SEP_SPECIAL_TOKEN = 102  # the input id for the sep special token that separates the question from the context. The context starts after this token. 
    PAD_SPECIAL_TOKEN = 0  # the input id for padding tokens added to the end of the context
    
    span_probabilities = []  # save the probabilities here
    spans = []  # save the possible answer spans here
    span_store = set() # new store for already seen spans
    
    for start_idx in range(0, input_length):
        for end_idx in range(start_idx, input_length):  # start the inner loop from start_idx to avoid start_idx > end_idx
            
            start_prob = probs_start[start_idx]
            end_prob = probs_end[end_idx]

            # Compute the span probability
            span_prob = start_prob * end_prob

            span = tokenizer.decode(input_ids[start_idx:end_idx+1])

            # exclude empty spans and whitespace spans and spans we seen before and span length > 1:
            if span and not span.isspace() and span != "" and span not in span_store and len(span)>1:
                span_store.add(span)
                spans.append(span)
                span_probabilities.append(span_prob)

    # list span and its probability
    span_probs = list(zip(spans, span_probabilities))
    span_probs.sort(key=lambda x: x[1], reverse=True) # sort them

    # loop and filter out longer spans that contain shorter spans with higher probabilities
    filtered_spans = []
    for span, prob in span_probs:
        if not any(s in span for s, p in filtered_spans if p > prob):
            filtered_spans.append((span, prob))

    # print the top 20 answers:
    for i in range(min(20, len(filtered_spans))):
        print(f'Span prob = {filtered_spans[i][1]}, answer = {filtered_spans[i][0]}')

predict_answer(probs_start[example_index], probs_end[example_index], example['input_ids'], tokenizer)     

Span prob = 0.5619066953659058, answer = endangeredriver
Span prob = 0.006627378053963184, answer = ##river
Span prob = 0.0014155315002426505, answer = Quick turns. Narrow
Span prob = 0.0013854469871148467, answer = endangered
Span prob = 0.0006158535834401846, answer = Quick turns
Span prob = 0.0002502755669411272, answer = Narrow
Span prob = 0.00021596302394755185, answer = The #
Span prob = 8.011087629711255e-05, answer = water
Span prob = 3.96909199480433e-05, answer = John D. Sutter
Span prob = 2.2990030629443936e-05, answer = The
Span prob = 1.8333874322706833e-05, answer = Quick
Span prob = 2.914824335675803e-06, answer = turns
Span prob = 2.4974292500701267e-06, answer = ##rrow
Span prob = 1.5744464008093928e-06, answer = ( I'm losing it )
Span prob = 1.3195754036132712e-06, answer = jdsutter
Span prob = 9.677168009147863e-07, answer = sexy bastard
Span prob = 7.168869728957361e-07, answer = John
Span prob = 5.012100245949114e-07, answer = June 21, 2014
Span prob = 4.5611028554

You can try out the pretrained QA model on a few examples and try to identify its common mistakes.

**TO-DO 5c:** State one way that we could improve the performance of our extractive QA model on the Tweet QA dataset.  **2 marks**

WRITE YOUR ANSWER HERE

## Our model has been trained on the Squad dataset which is different from the Tweet dataset. Very possible they have different statistical properties and differ in length and style. Therefore tuning to the Tweet QA is one possible way to improve. We could also adjust the tokenizer to be able to handle tweet features, like hastages or @mentions with a generic token and allow for slang/abbreviations.

--- 

As well as answering ad-hoc queries, question answering models can help us to extract structured information about entities of interest from a large set of documents. Suppose that we want to automatically collect information on tech companies, such as Apple and Open AI. We want to extract information about each company's activities from social media, including the names and release dates of new products and services, the company's earnings in a specific year, and who its CEO is.  

**TO-DO 5d:** Given a list of tech company names, how could we use question answering to extract the required information for each company from a set of tweets?  **(3 marks)** 

WRITE YOUR ANSWER HERE

## 1. Design a set of questions as noted above e.g. "who is the current CEO of Apple?", "how much profit did Apple make last year?"
## 2. Collect all the Tweets we can about these companies, not sure if it is still available but using the Twitter API for example.
## 3. Prepare the Tweets in the pipeline as discussed in previous answer, cleaning, tokenizing etc.
## 4. Run the tweet text and question through a QA model
## 5. Process the results as we have in 5b, the predict_answer function may need further adjustment
## 6. Check some of the results i.e. see if it is getting the current CEO correct since (most) Tweets may not be telling the truth. 
--- 

# 6. Transformer-based Text Classifiers (max. 24 marks)

The previous section showed us how to use a pretrained QA model based on a pretrained transformer. In this section, you will learn how to construct and train a text classifier on top of a pretrained transformer. 

We will use the [Poem Sentiment](https://huggingface.co/datasets/poem_sentiment) dataset to train and test a classifier. The task is to classify lines from poems into one of  0: negative, 1: positive, 2: no impact, or 3: mixed sentiment. For more information, see [Sheng and Uthus, 2020](https://arxiv.org/pdf/2011.02686.pdf). 

To begin you will need to instantiate a suitable model.

**TO-DO 6a:** Find an AutoModel class that constructs a text classifier from the pretrained TinyBERT model, "huawei-noah/TinyBERT_General_4L_312D". Create the `model` object in the cell below using this class. Refer to the [Hugging Face documentation for auto models](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html) as needed. **(2 marks)**

In [72]:
### WRITE YOUR ANSWER TO 6a HERE ###
from transformers import AutoModelForSequenceClassification

# classication model with 4 labels - 0: negative, 1: positive, 2: no impact, or 3: mixed sentiment
model = AutoModelForSequenceClassification.from_pretrained("huawei-noah/TinyBERT_General_4L_312D", num_labels=4)


Some weights of the model checkpoint at huawei-noah/TinyBERT_General_4L_312D were not used when initializing BertForSequenceClassification: ['fit_denses.0.bias', 'fit_denses.3.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'fit_denses.2.bias', 'fit_denses.1.bias', 'fit_denses.0.weight', 'fit_denses.4.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'fit_denses.2.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'fit_denses.4.weight', 'fit_denses.3.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'fit_denses.1.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a

**TO-DO 6b:** Provide a link to the documentation for your chosen auto model for text classification. Briefly describe how the text classifier `model` it creates differs from the QA model created by `AutoModelForQuestionAnswering`. Note: A useful reference may be the original BERT paper (https://arxiv.org/pdf/1810.04805.pdf), which includes diagrams (Figure 4) showing how BERT can be adapted to different tasks. **(2 marks)** 

WRITE YOUR ANSWER TO 6b HERE:

## Link to documentation = https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#tfautomodelforsequenceclassification

## As discussed in the BERT paper: "Apart from output layers, the same architectures are used in both pre-training and fine-tuning." The main difference in the models is in their final output layer and what they are used for. In Figure 4, the AutoModelForSequenceClassification corresponds to the ''sentence air classification'' where the output from the CLS token is fed into an output layer for classification. The QA model corresponds to the ''SQuAD' model, with the output layer predicting start, end, and span. The main structure of both models is the same.

---

For the QA task, the complete model was pretrained and we could apply it to a dataset without further training. However, for our poem sentiment classification task,
we will need to train our model before we can use it (you may see a message in the output of the last cell telling you this). 

**TO-DO 6c:** The emotion classifier is built on top of a pretrained TinyBERT model, so why do we need to train it before we can use it? **(2 marks)**

WRITE YOUR ANSWER TO 6C HERE:
## While the pre-trained TinyBERT model has learned a lot of useful language features during its pretraining phase on a large corpus of text, it needs to be fine-tuned explicitly for the emotion classification task.

## During pretraining, models like TinyBERT learn a generalised understanding of the language, including syntax, grammar, and of semantics. The process of fine-tuning adapts the model to a specific task, such as emotion classification. In the fine-tuning phase, the model learns from a labelled dataset where each text (i.e. a line from a poem) is associated with a specific class of emotion. The model learns to associate the language features extracted during pretraining with the specific task of predicting the emotion of a text to make it perform better in poetry.

---

Next, let's learn how to train our model. For some tasks it is not necessary to update the weights in the BERT model itself, so we can freeze them to save a lot of computation time. We can do this as follows. Since our pretrained model is based on BERT, we can access the weights inside BERT through the variable `model.bert`.

In [73]:
# frozen paramters
for param in model.bert.parameters():
    param.requires_grad = False

To train our model, we can make use of the Trainer class, which encapsulates a lot of the complex training steps and avoids the need to define our own training function, as we did in the previous notebook (we don't need to write our own `train_nn`).

First, define some settings for the training process. This is where we can set training hyperparameters:

In [74]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='transformer_checkpoints',     # specify the directory where models weights will be saved a certain points during training (checkpoints)
    num_train_epochs=10,              # A sensible and sufficient number to use for the to-dos below
    per_device_train_batch_size=6,    # was 8, you can decrease this if memory usage is too high while training
    logging_steps=50,                 # how often to print progress during training
)

Next, create a trainer object. Note that the next cell will currently fail with an error, because the variables `poem_train_dataset` and `poem_val_dataset` do not exist yet! Don't worry, we'll fix this later. 

from transformers import Trainer
from torch import nn

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=poem_train_dataset,
    eval_dataset=poem_val_dataset,
)

To train the model, you will need to call `trainer.train()`.

Once the model is trained, we can obtain predictions using the function below. Notice that it is simpler than obtaining the spans for QA -- we simply get the logits for each tweet in the test set, then apply argmax over the classes to find the most probable class for each tweet:

In [75]:
def predict_nn(trained_model, test_dataset):

    # Switch off dropout
    trained_model.eval()
    
    # Pass the required items from the dataset to the model    
    output = trained_model(attention_mask=torch.tensor(test_dataset["attention_mask"]), input_ids=torch.tensor(test_dataset["input_ids"]))
                        
    # the output dictionary contains logits, which are the unnormalised scores for each class for each example:
    pred_labs = np.argmax(output["logits"].detach().numpy(), axis=1)

    return pred_labs

You should now have all the bits and pieces needed to build and train a text classifier. Let's put them all together...

**TO-DO 6d:** Implement and test a classifier for the [Poem Sentiment](https://huggingface.co/datasets/poem_sentiment) dataset using a pretrained transformer. Evaluate the classifier with both frozen and unfrozen (i.e., fine-tuned) parameters in the pretrained transformer. Choose a suitable evaluation metric and provide a comparison of the results below, including a brief explanation  (1-2 sentences) for any differences you observe between the frozen and unfrozen variants. Make sure to comment your code.  **(10 marks)**

Notes: 
 * Strong classifier performance is not required to achive good marks -- rather, we award marks for implementing and testing a transformer-based classifier correctly.
 * You may implement any suitable kind of classifier you like, as long as you are using a pretrained transformer model.
 * 'tiny' BERT variants such as TinyBERT and roberta-tiny are recommended because they are small enough to fine-tune with a typical laptop CPU. We recommend sticking with these smaller pretrained models unless you have access to a GPU, e.g., via Google Colab. 

WRITE YOUR ANSWER HERE (DESCRIPTION OF RESULTS FOR 6d):


Have collected sample results and tehn summarised:

frozen results:
* eval_loss: 1.2594
* eval_runtime: 0.1026
* eval_samples_per_second: 1023.2430
* eval_steps_per_second: 136.4320

frozen results 2:
* eval_loss: 1.2588
* eval_runtime: 1.2839
* eval_samples_per_second: 81.7810
* eval_steps_per_second: 10.9040

unfrozen results:
* eval_loss: 0.9391
* eval_runtime: 0.1429
* eval_samples_per_second: 734.5760
* eval_steps_per_second: 97.9430

unfrozen results 2:
* eval_loss: 0.8033
* eval_runtime: 1.7196
* eval_samples_per_second: 61.0600
* eval_steps_per_second: 8.1410

Over ten epochs, we can see that the evaluation loss is more significant in the frozen model, which indicates that the unfrozen model made fewer errors. This suggests that allowing the parameters to update improves performance. 
The total evaluation runtime for the unfrozen model is longer, and it evaluates fewer samples per second; this highlights the additional computational requirements required for the more complex model.

Classification report for the model with frozen parameters, weighted avg:
* precision: 0.44    
* recall: 0.66
* f1-score: 0.53   
* support: 104


Classification report for the model with unfrozen parameters, weighted avg:
* precision: 0.87    
* recall: 0.86
* f1-score: 0.86   
* support: 104

The unfrozen model performs better in all categories.

In [86]:
### WRITE YOUR ANSWER HERE (Code for 6d; feel free to use multiple cells and copy code from above) ###

# sequence classifier model
from transformers import AutoModelForSequenceClassification

# classication model with 4 labels - 0: negative, 1: positive, 2: no impact, or 3: mixed sentiment
model = AutoModelForSequenceClassification.from_pretrained("huawei-noah/TinyBERT_General_4L_312D", num_labels=4)

# frozen parameters for first evaluation
for param in model.bert.parameters():
    param.requires_grad = False

from transformers import TrainingArguments

checkpoint_folder = 'transformer_checkpoints'

training_args = TrainingArguments(
    output_dir=checkpoint_folder,     # specify the directory where models weights will be saved a certain points during training (checkpoints)
    num_train_epochs = 10,              # A sensible and sufficient number to use for the to-dos below
    per_device_train_batch_size = 8,    # 6 in Jupyter, trying 8 in COLAB, you can decrease this if memory usage is too high while training
    logging_steps = 50,                 # how often to print progress during training
)

def predict_nn(trained_model, test_dataset):

    # Switch off dropout
    trained_model.eval()
    
    # Pass the required items from the dataset to the model    
    output = trained_model(attention_mask=torch.tensor(test_dataset["attention_mask"]), input_ids=torch.tensor(test_dataset["input_ids"]))
                        
    # the output dictionary contains logits, which are the unnormalised scores for each class for each example:
    pred_labs = np.argmax(output["logits"].detach().numpy(), axis=1)

    return pred_labs

# need to get the poem data from the cache
from datasets import load_dataset
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer

cache_dir = "./data_cache"

poem_train_dataset = load_dataset(
    "poem_sentiment",
    name="emotion",
    split="train",
    ignore_verifications=True,
    cache_dir=cache_dir,
)
print(f"Training dataset with {len(poem_train_dataset)} instances loaded")

poem_val_dataset = load_dataset(
    "poem_sentiment",
    name="emotion",
    split="validation",
    ignore_verifications=True,
    cache_dir=cache_dir,
)
print(f"Development/validation dataset with {len(poem_val_dataset)} instances loaded")

poem_test_dataset = load_dataset(
    "poem_sentiment",
    name="emotion",
    split="test",
    ignore_verifications=True,
    cache_dir=cache_dir,
)
print(f"Test dataset with {len(poem_test_dataset)} instances loaded")

# use the appropriate tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")

# function to apply tokenization to the dataset, don't need SEP as only one sentence, the poem line
def tokenize(batch):
    return tokenizer(batch['verse_text'], padding="max_length", max_length=50, truncation=True)

# tokenize datasets
poem_train_dataset = poem_train_dataset.map(tokenize, batched=True, batch_size=len(poem_train_dataset))
poem_val_dataset = poem_val_dataset.map(tokenize, batched=True, batch_size=len(poem_val_dataset))
poem_test_dataset = poem_test_dataset.map(tokenize, batched=True, batch_size=len(poem_test_dataset))

# format dataset to outputs torch.Tensor
poem_train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
poem_val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
poem_test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# training the model
from transformers import Trainer
from torch import nn

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=poem_train_dataset,
    eval_dataset=poem_val_dataset,
)

trainer.train()

# evaluate using the eval_loss
# evaluate frozen model on the validation dataset
eval_results = trainer.evaluate()

print('frozen results:')
for key, value in sorted(eval_results.items()):
    print(f"{key}: {value:.4f}")

# Save the model with frozen parameters
frozen_model = model

Some weights of the model checkpoint at huawei-noah/TinyBERT_General_4L_312D were not used when initializing BertForSequenceClassification: ['fit_denses.0.bias', 'fit_denses.3.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'fit_denses.2.bias', 'fit_denses.1.bias', 'fit_denses.0.weight', 'fit_denses.4.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'fit_denses.2.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'fit_denses.4.weight', 'fit_denses.3.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'fit_denses.1.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a

Training dataset with 892 instances loaded


Found cached dataset poem_sentiment (C:/Users/antro/OneDrive - University of Bristol/ADVANCED DA/Assignment/advanced-labs-public/data_cache/poem_sentiment/emotion/1.0.0/4e44428256d42cdde0be6b3db1baa587195e91847adabf976e4f9454f6a82099)


Development/validation dataset with 105 instances loaded


Found cached dataset poem_sentiment (C:/Users/antro/OneDrive - University of Bristol/ADVANCED DA/Assignment/advanced-labs-public/data_cache/poem_sentiment/emotion/1.0.0/4e44428256d42cdde0be6b3db1baa587195e91847adabf976e4f9454f6a82099)


Test dataset with 104 instances loaded


Loading cached processed dataset at C:\Users\antro\OneDrive - University of Bristol\ADVANCED DA\Assignment\advanced-labs-public\data_cache\poem_sentiment\emotion\1.0.0\4e44428256d42cdde0be6b3db1baa587195e91847adabf976e4f9454f6a82099\cache-9d44a77b48572ade.arrow
Loading cached processed dataset at C:\Users\antro\OneDrive - University of Bristol\ADVANCED DA\Assignment\advanced-labs-public\data_cache\poem_sentiment\emotion\1.0.0\4e44428256d42cdde0be6b3db1baa587195e91847adabf976e4f9454f6a82099\cache-efc53793cddcc9ff.arrow
Loading cached processed dataset at C:\Users\antro\OneDrive - University of Bristol\ADVANCED DA\Assignment\advanced-labs-public\data_cache\poem_sentiment\emotion\1.0.0\4e44428256d42cdde0be6b3db1baa587195e91847adabf976e4f9454f6a82099\cache-3b051eb1d40a63fb.arrow
C:\Users\antro\anaconda3\envs\data_analytics\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch im

frozen results:
epoch: 10.0000
eval_loss: 1.2529
eval_runtime: 0.9052
eval_samples_per_second: 115.9960
eval_steps_per_second: 15.4660


In [87]:
import numpy as np
import torch

from sklearn.metrics import classification_report

# predict_nn 
frozen_predictions = predict_nn(frozen_model, poem_test_dataset)

# true labels
true_labels = poem_test_dataset['label']

print("Classification report frozen parameters:")
print(classification_report(true_labels, frozen_predictions)) 

C:\Users\antro\AppData\Local\Temp\ipykernel_14788\2742405498.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = trained_model(attention_mask=torch.tensor(test_dataset["attention_mask"]), input_ids=torch.tensor(test_dataset["input_ids"]))


Classification report frozen parameters:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        19
           1       0.00      0.00      0.00        16
           2       0.66      1.00      0.80        69

    accuracy                           0.66       104
   macro avg       0.22      0.33      0.27       104
weighted avg       0.44      0.66      0.53       104



C:\Users\antro\anaconda3\envs\data_analytics\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\antro\anaconda3\envs\data_analytics\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\antro\anaconda3\envs\data_analytics\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [88]:
# Test the model
pred_labels = predict_nn(frozen_model, poem_test_dataset)

# Compute accuracy
accuracy = (pred_labels == true_labels.numpy()).mean()
print(f'Test Accuracy: {accuracy:.4f}')


C:\Users\antro\AppData\Local\Temp\ipykernel_14788\2742405498.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = trained_model(attention_mask=torch.tensor(test_dataset["attention_mask"]), input_ids=torch.tensor(test_dataset["input_ids"]))


Test Accuracy: 0.6635


In [79]:
# unfreeze the parameters
for param in model.parameters():
    param.requires_grad = True

# retrian the unfrozen model
unfrozen_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=poem_train_dataset,
    eval_dataset=poem_val_dataset,
)

unfrozen_trainer.train()

# Save the model with unfrozen parameters
unfrozen_model = model

C:\Users\antro\anaconda3\envs\data_analytics\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [80]:
# evaluate unfrozen
unfrozen_eval_results = unfrozen_trainer.evaluate()

print('unfrozen results:')
for key, value in sorted(unfrozen_eval_results.items()):
    print(f"{key}: {value:.4f}")

unfrozen results:
epoch: 10.0000
eval_loss: 0.8083
eval_runtime: 0.8536
eval_samples_per_second: 123.0040
eval_steps_per_second: 16.4010


In [81]:
# predict_nn function
unfrozen_predictions = predict_nn(unfrozen_model, poem_test_dataset)

# Get the true labels
true_labels = poem_test_dataset['label']

print("Classification report unfrozen parameters:")
print(classification_report(true_labels, unfrozen_predictions))


C:\Users\antro\AppData\Local\Temp\ipykernel_14788\2742405498.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = trained_model(attention_mask=torch.tensor(test_dataset["attention_mask"]), input_ids=torch.tensor(test_dataset["input_ids"]))


Classification report unfrozen parameters:
              precision    recall  f1-score   support

           0       0.73      1.00      0.84        19
           1       0.67      0.75      0.71        16
           2       0.95      0.81      0.87        69
           3       0.00      0.00      0.00         0

    accuracy                           0.84       104
   macro avg       0.59      0.64      0.61       104
weighted avg       0.87      0.84      0.84       104



C:\Users\antro\anaconda3\envs\data_analytics\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\antro\anaconda3\envs\data_analytics\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\antro\anaconda3\envs\data_analytics\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [82]:
# Test the model
pred_labels = predict_nn(unfrozen_model, poem_test_dataset)

# Compute accuracy
accuracy = (pred_labels == true_labels.numpy()).mean()
print(f'Test Accuracy: {accuracy:.4f}')

C:\Users\antro\AppData\Local\Temp\ipykernel_14788\2742405498.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = trained_model(attention_mask=torch.tensor(test_dataset["attention_mask"]), input_ids=torch.tensor(test_dataset["input_ids"]))


Test Accuracy: 0.8365


**TO-DO 6e:** Did your sentiment classifier make use of any kind of model transfer or transfer learning? If so, what kinds of transfer were used and what benefit do they provide? **(4 marks)**

WRITE YOUR ANSWER HERE:

Yes, the classifier is built on a pre-trained transformer model before being fine-tuned on the Poem Sentiment text data. The transformer model is pre-trained using language modelling on a large corpus of data allowing it to pick up a lot of meaning and sentiment regarding words. The transformer model parameters are already well-trained, saving much computational time. We then retrain these parameters to adjust for the specific task, using training data from the Poem set.

Freezing and unfreezing the parameters is a form of feature extraction. When the parameters are from, the weights of the pre-trained model are fixed, and only the output (classification in this case) layer is updated. When unfrozen, the pre-trained model weights are also adjusted.

---

**TO-DO 6f:** Use your model to compute the probability of sentiment for a sentence of your choosing. Comment your code and print the sentence with its probability distribution. Label the values so that we know which class they refer to. **(4 marks)**

Hint: you could use a poem generator, such as [this one](https://www.poemofquotes.com/tools/poetry-generator/ai-poem-generator), to generate a test sentence. 

In [83]:
# WRITE YOUR ANSWER HERE   
# From Friedrich Nietzsche's "Thus Spoke Zarathustra": 

In [84]:
import torch
from torch.nn.functional import softmax

# Define the sentences
sentences = [
    "Man is a rope, tied between beast and overman — a rope over an abyss",
    "What is great in man is that he is a bridge and not an end",
    "what can be loved in man is that he is an overture and a going under",
    "I love those who do not know how to live, except by going under, for they are those who cross over"
]

# Tokenize the sentences
inputs = tokenizer(sentences, padding=True, truncation=True, max_length=50, return_tensors="pt")

# inputs to the same device as the model
inputs = {name: tensor.to(model.device) for name, tensor in inputs.items()}

# Compute logits
with torch.no_grad():
    outputs = model(**inputs)

# probabilities
probabilities = softmax(outputs.logits, dim=-1)

# print sentence and probability distribution
for sentence, probs in zip(sentences, probabilities):
    print(f"Sentence: {sentence}")
    print(f"Probabilities: negative={probs[0]:.4f}, positive={probs[1]:.4f}, no impact={probs[2]:.4f}, mixed sentiment={probs[3]:.4f}\n")


Sentence: Man is a rope, tied between beast and overman — a rope over an abyss
Probabilities: negative=0.9252, positive=0.0030, no impact=0.0387, mixed sentiment=0.0331

Sentence: What is great in man is that he is a bridge and not an end
Probabilities: negative=0.0034, positive=0.9423, no impact=0.0114, mixed sentiment=0.0429

Sentence: what can be loved in man is that he is an overture and a going under
Probabilities: negative=0.0047, positive=0.9280, no impact=0.0047, mixed sentiment=0.0626

Sentence: I love those who do not know how to live, except by going under, for they are those who cross over
Probabilities: negative=0.0059, positive=0.8890, no impact=0.0169, mixed sentiment=0.0882

